# 2018-06-01 - Prédire un couple Theta/B_theta 

# Génération MC
Pour utiliser les deux propriétés il faut générer le couple theta_btheta directement dans le nom du folder et ensuite récupérer le label avec set.class :

In [30]:
import MotionClouds as mc
import numpy as np
import matplotlib.pyplot as plt
import os
import imageio

#MC params
downscale = 1
fx, fy, ft = mc.get_grids(mc.N_X/downscale, mc.N_Y/downscale, 1)

#generation params
N_folders = 50
N_mc = 5

b_thetas = np.pi*np.logspace(-7, 0, N_folders, base=2)

thetas = np.linspace(0, np.pi, 16, endpoint=False)


print('Btheta list : \n'+str( b_thetas))
print('theta list : \n'+str(thetas))

Btheta list : 
[0.02454369 0.02709843 0.0299191  0.03303336 0.03647179 0.04026812
 0.04445961 0.04908739 0.05419687 0.05983819 0.06606672 0.07294357
 0.08053624 0.08891921 0.09817477 0.10839373 0.11967639 0.13213344
 0.14588715 0.16107247 0.17783843 0.19634954 0.21678747 0.23935277
 0.26426688 0.2917743  0.32214494 0.35567685 0.39269908 0.43357494
 0.47870554 0.52853377 0.58354859 0.64428988 0.7113537  0.78539816
 0.86714988 0.95741109 1.05706754 1.16709719 1.28857976 1.4227074
 1.57079633 1.73429975 1.91482217 2.11413508 2.33419437 2.57715953
 2.84541481 3.14159265]
theta list : 
[0.         0.19634954 0.39269908 0.58904862 0.78539816 0.9817477
 1.17809725 1.37444679 1.57079633 1.76714587 1.96349541 2.15984495
 2.35619449 2.55254403 2.74889357 2.94524311]


In [40]:
path = 'clouds_couple'

print('Generating clouds...')
#loop through b_thetas and thetas
for i_btheta, B_theta in enumerate(b_thetas) :
    for i_theta, theta in enumerate(thetas) :
        if not os.path.exists('./%s/b%st%s' % (path, B_theta*180/np.pi, theta*180/np.pi)): #folders are of shape 0b0
            os.makedirs('./%s/b%st%s' % (path, B_theta*180/np.pi, theta*180/np.pi))
        
        mc_i = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0., B_V=0, theta=theta, B_theta=B_theta)
        im = mc.random_cloud(mc_i)
        im = (mc.rectif(im) * 255).astype('uint8')
        imageio.imwrite('./%s/b%st%s/%s.png' % (path, B_theta*180/np.pi , theta*180/np.pi, str('MC')),
                        im[:, :, 0])
print('Clouds generated !')

path = 'clouds_couple_test'
N_folders = 5

b_thetas = np.pi*np.logspace(-7, 0, N_folders, base=2)
thetas = np.linspace(0, np.pi, 16, endpoint=False)
print('Generating test set ...')
for i_btheta, B_theta in enumerate(b_thetas) :
    for i_theta, theta in enumerate(thetas) :
        if not os.path.exists('./%s/b%st%s' % (path, B_theta*180/np.pi, theta*180/np.pi)): #folders are of shape 0b0
            os.makedirs('./%s/b%st%s' % (path, B_theta*180/np.pi, theta*180/np.pi))
        
        mc_i = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0., B_V=0, theta=theta, B_theta=B_theta)
        im = mc.random_cloud(mc_i)
        im = (mc.rectif(im) * 255).astype('uint8')
        imageio.imwrite('./%s/b%st%s/%s.png' % (path, B_theta*180/np.pi , theta*180/np.pi, str('MC')),
                        im[:, :, 0])
print('Test set generated !')

Generating clouds...
Clouds generated !
Generating test set ...
Test set generated !


# Réseau convolutionnel
Maintenant on va tenter de prédire les couples en utilisant le réseau convolutionnel classique à 5 couches, utilisé en première semaine :

In [39]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch 
import torchvision
from torchvision import transforms, datasets

#Transform
data_transform = transforms.Compose(
    [transforms.Grayscale(),
     transforms.Resize((64,64)),
    transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5,0.5), (0.5,0.5,0.5))])


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 10, 5)
        self.fc1 = nn.Linear(1690, 1000)
        self.fc2 = nn.Linear(1000, 100)
        self.fc3 = nn.Linear(100, 20)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

model = Net()
print(model)

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 10, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=1690, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=20, bias=True)
)


In [45]:
import torch.optim as optim

#Criterion
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

#Train
train_set = datasets.ImageFolder(root='clouds_couple',
                                transform=data_transform)
train_loader = torch.utils.data.DataLoader(train_set,
                                             batch_size=8, shuffle=False,
                                             num_workers=1)
#Test
test_set = datasets.ImageFolder(root='clouds_couple_test',
                                transform=data_transform)
test_loader = torch.utils.data.DataLoader(test_set,
                                             batch_size=8,shuffle=False,
                                             num_workers=1)

#Plotting lists
total_step = len(train_loader)
loss_list = []
time_list = []
t=0

#Hyperparameters
epochs = 10

import time
start_time = time.time()
print("Started training")

for epoch in range(epochs) :
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, epochs, i+1, total_step, loss.data[0]))
            loss_list.append(loss.data[0])
            time_list.append(t)
            t+=1

Started training
tensor([ 0,  1,  2,  3,  4,  5,  6,  7])
tensor([  8,   9,  10,  11,  12,  13,  14,  15])
tensor([ 16,  17,  18,  19,  20,  21,  22,  23])


RuntimeError: Assertion `cur_target >= 0 && cur_target < n_classes' failed.  at /opt/conda/conda-bld/pytorch_1524590031827/work/aten/src/THNN/generic/ClassNLLCriterion.c:97